In [47]:
import pandas as pd
import numpy as np

In [48]:
!git clone https://github.com/9berny9/bikes

fatal: destination path 'bikes' already exists and is not an empty directory.


# DATABASE BIKES

Tento colab notebook obsahuje práci a úpravu dat na vstupech z databáze.

# OBSAH
1. [Jednotlivé datasety](#1)
      1. [Bikes](#1.1)
      2. [Weather](#1.2)
      3. [Stations](#1.3)
2. [Sjednocený dataset](#2)



### Jednotlivé datasety <a name="1"></a>

Pro zjednodušení a prezentaci práce si vytvořím pickle databáze, s kterým budu pracovat. V reálném projektu bych pracoval přímo s databází.

Postup:
1.   Načtení databáze do jednotlivých dataframů -> `data = pd.read_sql(sql=query, con=engine)`
2.   Uložení dataframů jako pickle -> `data.to_pickle("data_basic.pkl")`
3.   Načtení do dataframu + kontrola datových formátů, chybějící údajů, řádků i sloupců a jejich následná úprava.
4.   Upravený dataframe uložení do nového pickle souboru.

#### Bikes <a name="2.1.1"></a>

Databáze bikes obsahuje informace o datumu a času vypůjčení kol s iformacemi o místě převzetí kola a jeho vrácení.

In [49]:
# url github clonovaného repositáře v Colabu
url_bikes_basic = "/content/bikes/Database/bikes_first.pkl"

In [50]:
# načtení do dataframu
dfb = pd.read_pickle(url_bikes_basic)

In [51]:
dfb.head()

,index,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653
1,1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.954749,-3.192774,262,Canonmills,near Tesco's,55.962804,-3.196284
2,2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.962804,-3.196284,250,Victoria Quay,Entrance to Scottish Government Office,55.977638,-3.174116
3,3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254,Kings Building 3,Kings Building House,55.923479,-3.175385
4,4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,253,Kings Building 2,Sanderson Building,55.923202,-3.171646


In [52]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438259 entries, 0 to 438258
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   index                      438259 non-null  int64  
 1   started_at                 438259 non-null  object 
 2   ended_at                   438259 non-null  object 
 3   duration                   438259 non-null  int64  
 4   start_station_id           438259 non-null  int64  
 5   start_station_name         438259 non-null  object 
 6   start_station_description  435549 non-null  object 
 7   start_station_latitude     438259 non-null  float64
 8   start_station_longitude    438259 non-null  float64
 9   end_station_id             438259 non-null  int64  
 10  end_station_name           438259 non-null  object 
 11  end_station_description    435256 non-null  object 
 12  end_station_latitude       438259 non-null  float64
 13  end_station_longitude      43

Úprava datového typu u sloupců ***started_at*** a ***ended_at*** na formát datetime64 pro lepší práci s datem a časem.

In [53]:
# datový formát datetime64
dfb['started_at'] = pd.to_datetime(dfb['started_at'], format="%Y-%m-%d %H:%M")
dfb['ended_at'] = pd.to_datetime(dfb['ended_at'], format="%Y-%m-%d %H:%M")

In [54]:
# přejmenování sloupců na kratší názvy
dfb = dfb.rename(columns={"started_at": "start", "ended_at": "end",
        "start_station_id": "start_id", "start_station_name": "start_name",
        "start_station_description": "start_description",
        "start_station_latitude": "start_latitude",
        "start_station_longitude": "start_longitude","end_station_id": "end_id",
        "end_station_name": "end_name", 
        "end_station_description": "end_description",
        "end_station_latitude": "end_latitude",
        "end_station_longitude": "end_longitude"
        })

In [55]:
# odstranění sloupce
dfb = dfb.drop(columns= "index")

In [56]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438259 entries, 0 to 438258
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   start              438259 non-null  datetime64[ns]
 1   end                438259 non-null  datetime64[ns]
 2   duration           438259 non-null  int64         
 3   start_id           438259 non-null  int64         
 4   start_name         438259 non-null  object        
 5   start_description  435549 non-null  object        
 6   start_latitude     438259 non-null  float64       
 7   start_longitude    438259 non-null  float64       
 8   end_id             438259 non-null  int64         
 9   end_name           438259 non-null  object        
 10  end_description    435256 non-null  object        
 11  end_latitude       438259 non-null  float64       
 12  end_longitude      438259 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(3), 

In [57]:
# zjištění prvnícho záznamu
dfb["start"].min()

Timestamp('2018-09-15 08:52:05')

In [58]:
# zjištění posledního záznamu
dfb["start"].max()

Timestamp('2021-06-30 23:58:33')

In [59]:
# Uložení upraveného dataframu do pickle.
url_bikes = "/content/bikes/Database/bikes.pkl"
dfb.to_pickle(url_bikes)

Dataframe obsahuje **438 259** záznamů a **14** sloupců s informacemi od data **2018-09-15** do **2021-06-30**.

#### Weather <a name="2.1.2"></a>

Databáze Weather obsahuje informace o datumu a času měření údajů o počasí v městě Edinburgh. Měření jsou prováděny každé 3 hodiny.

In [60]:
# Načtení pickle do Dataframu.
url_weather = "/content/bikes/Database/weather_first.pkl"
dfw = pd.read_pickle(url_weather)

In [61]:
dfw.head()

,time,temp,feels,wind,gust,rain,humidity,cloud,pressure,vis,date
0,00:00,11 °c,11 °c,9 km/h from S,19 km/h,0.0 mm,79%,13%,1020 mb,Excellent,2018-09-01
1,03:00,13 °c,12 °c,11 km/h from SSW,19 km/h,0.0 mm,76%,96%,1020 mb,Excellent,2018-09-01
2,06:00,14 °c,13 °c,11 km/h from SSW,19 km/h,0.0 mm,84%,100%,1020 mb,Excellent,2018-09-01
3,09:00,14 °c,13 °c,14 km/h from SSW,23 km/h,0.1 mm,88%,78%,1021 mb,Excellent,2018-09-01
4,12:00,16 °c,16 °c,15 km/h from SSW,22 km/h,0.0 mm,87%,87%,1021 mb,Excellent,2018-09-01


In [62]:
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6336 entries, 0 to 6335
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   time      6336 non-null   object
 1   temp      6336 non-null   object
 2   feels     6336 non-null   object
 3   wind      6336 non-null   object
 4   gust      6336 non-null   object
 5   rain      6336 non-null   object
 6   humidity  6336 non-null   object
 7   cloud     6336 non-null   object
 8   pressure  6336 non-null   object
 9   vis       6336 non-null   object
 10  date      6336 non-null   object
dtypes: object(11)
memory usage: 544.6+ KB


In [63]:
# sjednocení datumu s časem a převedení na datetime64
dfw['time'] = pd.to_datetime((dfw['date'] + ' ' + dfw['time']),
                             format="%Y-%m-%d %H:%M" )

In [64]:
# vyjmutí jednotek ze sloupců a nastavení datového formátu
dfw["temp"] = dfw["temp"].str.extract('(\d+)').astype(int)
dfw['wind'] = dfw['wind'].str.extract('(\d+)').astype(int)
dfw["gust"] = dfw["gust"].str.extract('(\d+)').astype(int)
dfw["rain"] = dfw["rain"].str.extract('(\d+)').astype(float)

In [65]:
# odebrání nepotřebných sloupců
dfw = dfw.drop(columns=["feels","humidity","cloud","pressure","vis", "date"])

In [66]:
# přejmenování sloupců na název a jednotku
dfw = dfw.rename(columns={"time":"datetime", "temp":"temp_c",
                          "wind":"wind_kmh", "gust":"gust_kmh",
                          "rain":"rain_mm"})

In [67]:
dfw.head()

,datetime,temp_c,wind_kmh,gust_kmh,rain_mm
0,2018-09-01 00:00:00,11,9,19,0.0
1,2018-09-01 03:00:00,13,11,19,0.0
2,2018-09-01 06:00:00,14,11,19,0.0
3,2018-09-01 09:00:00,14,14,23,0.0
4,2018-09-01 12:00:00,16,15,22,0.0


In [68]:
# první záznam o počasí
dfw['datetime'].min()

Timestamp('2018-09-01 00:00:00')

In [69]:
# # poslední záznam o počasí
dfw['datetime'].max()

Timestamp('2020-10-31 21:00:00')

In [70]:
# Uložení upraveného dataframu do pickle.
dfw.to_pickle("/content/bikes/Database/weather.pkl")

Upravený dataframe obsahuje **6336** záznamů a 5 sloupců s informacemi od data **2018-09-01 00:00** do **2020-10-31 21:00**.

#### Stations <a name="**2.1.3**"></a>

* Z upraveného dataframu **bikes.pkl** si vytvořím dataframe pouze s unikátními stanicemi bez časových záznamů. Tento dataframe budu využívat jako seznam všech stanic s informacemi o ních.  
* K datasetu přidám sloupec s početem výpujček a vrácení kol v jednotlivých stanicích.
* Stanice, které jsou k dispozici zjistíme ze **start_id** a **end_id**

In [71]:
# načtení pickle do dataframu
url_bikes = "/content/bikes/Database/bikes.pkl"
dfs = pd.read_pickle(url_bikes)

In [72]:
dfs

,start,end,duration,start_id,start_name,start_description,start_latitude,start_longitude,end_id,end_name,end_description,end_latitude,end_longitude
0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653
1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.954749,-3.192774,262,Canonmills,near Tesco's,55.962804,-3.196284
2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.962804,-3.196284,250,Victoria Quay,Entrance to Scottish Government Office,55.977638,-3.174116
3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254,Kings Building 3,Kings Building House,55.923479,-3.175385
4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,253,Kings Building 2,Sanderson Building,55.923202,-3.171646
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438254,2021-06-30 23:30:31,2021-07-01 00:06:10,2139,1090,Hillside Crescent,East end of Hillside Crescent,55.957872,-3.175888,1728,Portobello - Kings Road,Foot of Kings Road next to the promenade,55.957915,-3.118332
438255,2021-06-30 23:36:16,2021-07-01 00:05:40,1763,1814,Abbeyhill,Near Abbey Mount,55.955248,-3.172216,1728,Portobello - Kings Road,Foot of Kings Road next to the promenade,55.957915,-3.118332
438256,2021-06-30 23:49:03,2021-07-01 00:11:25,1342,256,St Andrews House,beside Jacobs ladder,55.953164,-3.181682,1091,Holyrood Road,Opposite St Leonards Land,55.949560,-3.180413
438257,2021-06-30 23:49:03,2021-07-01 00:11:52,1369,256,St Andrews House,beside Jacobs ladder,55.953164,-3.181682,1091,Holyrood Road,Opposite St Leonards Land,55.949560,-3.180413


In [73]:
# odstranění nepotřebných sloupců
dfs_start = dfs.drop(columns=["start","end","duration","end_id","end_name",
                              "end_description","end_latitude","end_longitude"])

In [74]:
# vytvoření unikátních záznamů
dfs_start = dfs_start.drop_duplicates('start_id').reset_index(drop=True)

In [75]:
# přejmenování sloupců
dfs_start = dfs_start.rename(
            columns={"start_name": "name",
                     "start_id": "id",
                 "start_description": "description",
                 "start_latitude": "lat",
                 "start_longitude": "lon"})

In [76]:
# odstranění nepotřebných sloupců
dfs_end = dfs.drop(columns=["start","end","duration","start_id","start_name","start_description","start_latitude","start_longitude"])

In [77]:
# vytvoření unikátních záznamů
dfs_end = dfs_end.drop_duplicates('end_id').reset_index(drop=True)

In [78]:
# přejmenování sloupců
dfs_end = dfs_end.rename(
            columns={"end_name": "name",
                     "end_id": "id",
                 "end_description": "description",
                 "end_latitude": "lat",
                 "end_longitude": "lon"})

In [79]:
# sloučení dataframů
dfs_new = dfs_start.append(dfs_end, ignore_index = True)

In [80]:
# vytvoření unikátních záznamů
dfs_new = dfs_new.drop_duplicates('id').reset_index(drop=True)

In [81]:
# počet stanic
dfs_new["id"].count()

200

Nový dataframe, který obsahuje všechny bikes stanice obsahuje **200 stanic**.

In [82]:
# vytvoření nového sloupce s počtem vypůjčených kol ve stanici
start_frequency = dfs["start_id"].value_counts().rename_axis("id").reset_index(name = "start_frequency")

In [83]:
# vytvoření nového sloupce s počtem vrácených kol ve stanici
end_frequency = dfs["end_id"].value_counts().rename_axis("id").reset_index(name = "end_frequency")

In [84]:
# sjednocení dataframů
dfs_final = dfs_new.merge(start_frequency, how="outer", on = "id")
dfs_final = dfs_final.merge(end_frequency, how= "outer", on= "id")

In [86]:
dfs_final.head()

,id,name,description,lat,lon,start_frequency,end_frequency
0,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,8134.0,4980.0
1,259,St Andrew Square,North East corner,55.954749,-3.192774,12218.0,8372.0
2,262,Canonmills,near Tesco's,55.962804,-3.196284,9551.0,11940.0
3,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254.0,196.0
4,253,Kings Building 2,Sanderson Building,55.923202,-3.171646,4096.0,4557.0


In [87]:
# zjištění stanic, ve kterých nebylo nikdy zapůjčeno kolo
dfs_final[dfs_final["start_frequency"].isna()]

,id,name,description,lat,lon,start_frequency,end_frequency
198,242,Virtual Depot,Virtual Depot,55.972402,-3.155794,NaN,2.0
199,280,Smarter Travel Station,The Street,53.395525,-2.990138,NaN,3.0


In [88]:
# zjištění stanic, ve kterých nebylo nikdy vráceno kolo
dfs_final[dfs_final["end_frequency"].isna()]

,id,name,description,lat,lon,start_frequency,end_frequency
182,1857,City Chambers Launch Station,Temporary station at City Chambers,55.950222,-3.19027,1.0,NaN


In [89]:
# uložení sjednoceného dataframu jako pickle
dfs_final.to_pickle("/content/bikes/Database/stations.pkl")

V Edinburghu se nachází **200** stanic pro půjčení kol.

#### Sjednocený dataset <a name="2.2"></a>

Vytvoření sjednoceného datasetu z upravených datasetů o počasí a výpujčkách kol.

In [90]:
# Načtení dat
dfb1 = pd.read_pickle("/content/bikes/Database/bikes.pkl")
dfw2 = pd.read_pickle("/content/bikes/Database/weather.pkl")

In [91]:
# sjednocení dat o výpujčkách kol s daty o počasí. 
# Dataframe bikes omezím pouze na dostupné údaje o počasí.
last = str(dfw2.datetime.max())
merged_df = pd.merge_asof(dfb1[dfb1["start"] <= last], dfw2,
                                 left_on="start", right_on="datetime", 
                                 direction='nearest',
                                 allow_exact_matches=False).drop(
                                     columns="datetime")

In [92]:
merged_df.head()

,start,end,duration,start_id,start_name,start_description,start_latitude,start_longitude,end_id,end_name,end_description,end_latitude,end_longitude,temp_c,wind_kmh,gust_kmh,rain_mm
0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653,10,16,23,0.0
1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.954749,-3.192774,262,Canonmills,near Tesco's,55.962804,-3.196284,10,16,23,0.0
2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.962804,-3.196284,250,Victoria Quay,Entrance to Scottish Government Office,55.977638,-3.174116,10,16,23,0.0
3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254,Kings Building 3,Kings Building House,55.923479,-3.175385,14,32,47,0.0
4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,253,Kings Building 2,Sanderson Building,55.923202,-3.171646,14,32,47,0.0


In [93]:
# uložení jako pickle
merged_df.to_pickle("/content/bikes/Database/merged.pkl")